In [1]:
import dis
import inspect
from functools import wraps
from textwrap import dedent

from codetransformer.utils.pretty import a, d

def show_ast(s):
    a(dedent(s))
    
def show_disassembly(s):
    d(dedent(s))

import pytenn2016
from pytenn2016 import diagrams
from pytenn2016.tokens import show_tokens

diagrams.draw_all()

LookupError: 'rot13' is not a text encoding; use codecs.open() to handle arbitrary codecs

<center>
  <h1>Unspeakably Evil Hacks in Service of Marginally-Improved Syntax</h1><br><br>
  <h2>Compile-Time Metaprogramming in Python</h2><br><br>
  <h3><a href="https://github.com/ssanderson/pytenn2016">https://github.com/ssanderson/pytenn2016</a>
  </h3><br>
  
</center>

# Outline

- 5-minute Intro to "Standard" Metaprogramming
- Intro to CPython Compiler
- Import Hooks
- AST Transformers
- Bytecode Transformers

# Metaprogramming

<img alt="Xzibit loves metaprogramming!" src="images/yo-dawg.jpg" style="width: 1000px"/>

# Decorators

In [ ]:
def print_inputs(f):
    "A decorator that prints inputs to a function before calling it."
    @wraps(f)
    def print_then_call_f(*args, **kwargs):
        print("Args: %s" % (args,))
        print("Kwargs: %s" % kwargs)
        f(*args, **kwargs)
    return print_then_call_f

In [ ]:
@print_inputs
def my_func(a, b, c):
    print("Entering ``my_func``")
    print(a, b, c)
    print("Exiting ``my_func``")
    
my_func(1, 2, c=5)

# Metaclasses

In [ ]:
import inspect
import math

property_signature = inspect.FullArgSpec(
    args=['self'], 
    varargs=None, 
    varkw=None, 
    defaults=None, 
    kwonlyargs=[], 
    kwonlydefaults=None, 
    annotations={},
)


class AutoPropertyMeta(type):
    """
    A metaclass that wraps all no-argument methods of a subtype in properties.
    """
    def __new__(mcls, name, bases, clsdict):
        for name, class_attr in clsdict.items():
            try:
                signature = inspect.getfullargspec(class_attr)
            except TypeError: # Not everything in the classdict has to be callable.
                continue
            # Wrap anything with the right signature to be a property.
            if signature == property_signature:
                clsdict[name] = property(class_attr)

        return super().__new__(mcls, name, bases, clsdict)


In [ ]:
class Vector(metaclass=AutoPropertyMeta):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def magnitude(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    def doubled(self):
        return Vector(self.x * 2, self.y * 2)

In [ ]:
# Look mom, no parens!
print("Size: %s" % Vector(1, 2).magnitude)
print("Doubled Size: %s" % Vector(1, 2).doubled.magnitude)

# `exec`

## TODO

![Metaprogramming is the best](images/unicorn.jpg)

# That's all great but...

- Abstractions often incur runtime overhead.

- Certain operators can't be overloaded (e.g. ``is`` and ``not``).

- No support for syntactic extensions.
  - Can't add new syntax.
  - Often can't repurpose existing syntax.

<img alt="We have to go deeper!" src="images/leo.jpg" style="width: 1000px"/>

In [ ]:
# What happens when I hit Enter here?
def add1(a):
    return a + 1

add1(1)

![CPython Compiler Phases](images/compiler.svg)

# Custom Source Decoders

![decoder](images/decoder.svg)

In [2]:
import inspect
inspect.getsourcelines(pytenn2016.rot13)

NameError: name 'pytenn2016' is not defined

In [ ]:
show_disassembly(
    """\
    def add1(a):
        return a + 1
    add1(1)
    """
)

In [ ]:
add1.__code__

In [ ]:
add1.__code__.co_argcount

In [ ]:
add1.__code__.co_flags

In [ ]:
# ?!?!??
add1.__code__.co_lnotab

In [ ]:
add1.__defaults__ = (5,)

In [ ]:
add1()